# Alpha101 因子研究 Notebook

使用 `StockUniverse` 加载多股票数据集，通过 `Alpha101Indicators.from_dataset()` 进行**截面面板计算**，比单股票模式更精确（rank 等运算横跨全部股票）。

**流程：**
1. `StockUniverse` 定义标的池 → `.load()` 构建 `BacktestDataset`
2. `Alpha101Indicators.from_dataset(dataset)` 构建面板引擎
3. `engine.compute_factors([1, 6, 12, ...])` 批量计算指定因子
4. 将因子结果合并回每只股票的 DataFrame，可视化

> **提示：** 如果本地数据库为空，请先运行：
> ```bash
> cd stockquant/data && python3 -m updater --mode codes --codes 600519 000333 601899 002594 600276
> ```

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from stockquant.data.universe import Pool, StockUniverse
from stockquant.indicators import Alpha101Indicators
from stockquant.visualization.plot import PlotEngine

# ---------- 标的池配置 ----------
STOCKS = {
    "600519": "贵州茅台",
    "000333": "美的集团",
    "601899": "紫金矿业",
    "002594": "比亚迪",
    "600276": "恒瑞医药",
}

DATE_START = "2025-01-01"
DATE_END   = "2026-02-13"

# 要研究的 Alpha 因子
ALPHA_IDS = [1, 6, 12, 33, 41, 55, 101]

# 可视化引擎
plot_engine = PlotEngine(backend="plotly")

print(f"✅ 配置完成: {len(STOCKS)} 只股票, {len(ALPHA_IDS)} 个 Alpha 因子")

✅ 配置完成: 5 只股票, 7 个 Alpha 因子


## 1. 加载数据集 & 计算 Alpha 因子

In [13]:
# ---- 1. 用 StockUniverse 加载数据集 ----
dataset = (
    StockUniverse()
    .scope(list(STOCKS.keys()))          # 5 只指定股票
    .load(DATE_START, DATE_END, benchmark=Pool.CSI300)
)
print(dataset.summary())

# ---- 2. 构建面板引擎（截面 rank 覆盖全部股票）----
engine = Alpha101Indicators.from_dataset(dataset)

# ---- 3. 批量计算指定因子，返回 dict[int, DataFrame(date × code)] ----
factors: dict[int, pd.DataFrame] = engine.compute_factors(ALPHA_IDS)

# ---- 4. 将因子结果合并回每只股票的 DataFrame，保持可视化兼容 ----
stock_data: dict[str, pd.DataFrame] = {}
for code in dataset.codes:
    df = dataset.stock_data[code].copy()
    df["date"] = pd.to_datetime(df["date"])
    df = df.set_index("date").sort_index()
    for alpha_id, panel in factors.items():
        col = f"alpha{alpha_id:03d}"
        if code in panel.columns:
            df[col] = panel[code].reindex(df.index)
    stock_data[code] = df
    alpha_cols = [c for c in df.columns if c.startswith("alpha")]
    print(f"✅ {code} {STOCKS.get(code, '')}: {len(df)} bars, {len(alpha_cols)} 个因子")

print(f"\n共加载 {len(stock_data)} 只股票")

2026-02-22 08:45:47.606 | INFO     | stockquant.data.universe:load:377 | 加载标的池: 自定义(5只) → 5 只
2026-02-22 08:45:47.623 | INFO     | stockquant.data.database:conn:46 | 已连接 DuckDB: /workspaces/stockQuant/stockquant/data/db/stockquant.duckdb
2026-02-22 08:45:47.634 | INFO     | stockquant.data.database:init_tables:110 | 数据库表初始化完成
2026-02-22 08:45:47.635 | INFO     | stockquant.data.data_manager:__init__:43 | 数据管理器初始化完成，主数据源: akshare
2026-02-22 08:45:47.676 | INFO     | stockquant.data.universe:load:408 | 日线加载完成: 5/5 只
2026-02-22 08:45:47.680 | INFO     | stockquant.data.universe:_load_benchmark:549 | 基准 000300: 273 条（本地）
2026-02-22 08:45:47.706 | INFO     | stockquant.data.database:conn:46 | 已连接 DuckDB: /workspaces/stockQuant/stockquant/data/db/stockquant.duckdb
2026-02-22 08:45:47.725 | INFO     | stockquant.indicators.alpha101.alpha101:_auto_load_stock_info:418 | 已从 stock_info 加载行业数据: 5 个行业
2026-02-22 08:45:47.726 | INFO     | stockquant.indicators.alpha101.alpha101:_auto_load_stock_info

📊 回测数据集摘要
   标的数量: 5 只（共请求 5 只）
   日期范围: 2025-01-01 ~ 2026-02-13
   基准指数: 000300（273 条）
   日线总条数: 1,365
✅ 600519 贵州茅台: 273 bars, 7 个因子
✅ 000333 美的集团: 273 bars, 7 个因子
✅ 601899 紫金矿业: 273 bars, 7 个因子
✅ 002594 比亚迪: 273 bars, 7 个因子
✅ 600276 恒瑞医药: 273 bars, 7 个因子

共加载 5 只股票


## 2. 可视化工具函数

每个 Alpha 因子画一组图，每只股票一行，包含：
- **上方**：K 线图 + 成交量柱状图
- **下方**：Alpha 因子值曲线（带零轴线）

In [3]:
# Alpha 因子公式描述
ALPHA_DESC = {
    1:   "rank(Ts_ArgMax(SignedPower(cond, 2), 5)) − 0.5",
    6:   "−1 × correlation(open, volume, 10)",
    12:  "sign(delta(volume, 1)) × (−1 × delta(close, 1))",
    33:  "rank(−(1 − open / close))",
    41:  "(high × low)^0.5 − vwap",
    55:  "−corr(rank((close−ts_min(low,12))/(ts_max(high,12)−ts_min(low,12))), rank(volume), 6)",
    101: "(close − open) / ((high − low) + 0.001)",
}

# 配色
COLORS = ["#e74c3c", "#3498db", "#2ecc71", "#f39c12", "#9b59b6"]

def plot_alpha_group(alpha_id: int):
    """为一个 Alpha 因子画出 5 只股票的 K 线 + 成交量 + 因子值对比图。

    每只股票 3 行: K线 | 成交量 | Alpha 因子值。
    K 线 + 成交量通过 PlotEngine.build_kline_traces() 复用已有绘图逻辑。
    """
    col = f"alpha{alpha_id:03d}"
    n = len(stock_data)
    rows_per_stock = 3  # K线, 成交量, Alpha

    fig = make_subplots(
        rows=n * rows_per_stock, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.012,
        row_heights=[3, 1, 1.5] * n,
        subplot_titles=[
            item
            for code in stock_data
            for item in (f"{STOCKS[code]}({code}) K线", "成交量", f"Alpha#{alpha_id}")
        ],
    )

    for i, (code, df) in enumerate(stock_data.items()):
        row_candle = i * rows_per_stock + 1
        row_volume = i * rows_per_stock + 2
        row_alpha  = i * rows_per_stock + 3
        color = COLORS[i % len(COLORS)]
        dates = df.index

        # ---- K 线 + 成交量: 复用 PlotEngine ----
        kline_df = df.reset_index().rename(columns={df.index.name or "index": "date"})
        kline_traces = plot_engine.build_kline_traces(kline_df, name=STOCKS[code])
        for trace, target in kline_traces:
            row = row_candle if target == "candle" else row_volume
            fig.add_trace(trace, row=row, col=1)

        # ---- Alpha 因子 ----
        alpha_vals = df[col] if col in df.columns else pd.Series(np.nan, index=dates)

        fig.add_trace(
            go.Scatter(
                x=dates, y=alpha_vals,
                mode="lines",
                line=dict(color=color, width=1.2),
                fill="tozeroy",
                fillcolor=f"rgba({int(color[1:3],16)},{int(color[3:5],16)},{int(color[5:7],16)},0.15)",
                name=f"Alpha#{alpha_id}",
                showlegend=(i == 0),
            ),
            row=row_alpha, col=1,
        )

        # 零轴线
        fig.add_hline(y=0, line_dash="dot", line_color="gray",
                      line_width=0.8, row=row_alpha, col=1)

    desc = ALPHA_DESC.get(alpha_id, "")
    fig.update_layout(
        title=dict(
            text=f"Alpha#{alpha_id}  —  {desc}",
            font=dict(size=16),
        ),
        height=320 * n,
        width=1100,
        template="plotly_white",
        xaxis_rangeslider_visible=False,
        showlegend=True,
        margin=dict(l=60, r=30, t=60, b=30),
    )

    # 隐藏所有 rangeslider
    for ax_key in [k for k in fig.layout.to_plotly_json() if k.startswith("xaxis")]:
        fig.layout[ax_key]["rangeslider"] = dict(visible=False)

    fig.show()

print("✅ 绑定可视化函数 plot_alpha_group()")

✅ 绑定可视化函数 plot_alpha_group()


## 3. 因子分布概览

先看一下各因子在 5 只股票上的基本统计量。

In [4]:
rows = []
for code, df in stock_data.items():
    for aid in ALPHA_IDS:
        col = f"alpha{aid:03d}"
        if col in df.columns:
            s = df[col].dropna()
            rows.append({
                "股票": f"{STOCKS[code]}({code})",
                "Alpha": f"#{aid}",
                "均值": f"{s.mean():.4f}",
                "标准差": f"{s.std():.4f}",
                "最小值": f"{s.min():.4f}",
                "最大值": f"{s.max():.4f}",
                "有效率": f"{len(s)/len(df)*100:.1f}%",
            })

summary = pd.DataFrame(rows)
summary.style.set_properties(**{"text-align": "right"})

,股票,Alpha,均值,标准差,最小值,最大值,有效率
0,贵州茅台(600519),#1,0.0835,0.2468,-0.3000,0.5000,98.9%
1,贵州茅台(600519),#6,0.0899,0.3688,-0.7389,0.7646,98.5%
2,贵州茅台(600519),#12,-9.9020,90.2380,-572.9800,346.1100,99.6%
3,贵州茅台(600519),#33,0.6147,0.2481,0.2000,1.0000,100.0%
4,贵州茅台(600519),#41,2.2490,14.2027,-70.4290,65.8258,100.0%
5,贵州茅台(600519),#55,-0.0397,0.4460,-0.8333,1.2247,27.8%
6,贵州茅台(600519),#101,-0.0822,0.5265,-1.0000,0.9508,100.0%
7,美的集团(000333),#1,0.1150,0.2494,-0.3000,0.5000,99.3%
8,美的集团(000333),#6,0.0649,0.3566,-0.7803,0.8411,98.5%
9,美的集团(000333),#12,-0.1138,3.6118,-11.6600,23.1800,99.6%


## 4. 逐因子可视化

每个 Alpha 因子一组图，5 只股票纵向排列：上方 K 线，下方因子值。

### Alpha #1 — 波动率/价格条件排名因子
`rank(Ts_ArgMax(SignedPower(cond, 2), 5)) − 0.5`
- 当收益率 < 0 时使用波动率，否则使用收盘价，取有符号平方后找 5 日窗口内最大值位置

In [5]:
plot_alpha_group(1)

### Alpha #6 — 开盘价与成交量反相关因子
`−1 × correlation(open, volume, 10)`
- 10 日滚动窗口内 open 与 volume 的相关系数取反；当价量背离时信号较强

In [6]:
plot_alpha_group(6)

### Alpha #12 — 量价动量因子
`sign(delta(volume, 1)) × (−1 × delta(close, 1))`
- 量增价跌为正、量增价涨为负，捕捉量价背离的短期反转信号

In [7]:
plot_alpha_group(12)

### Alpha #33 — 开收盘价比率排名因子
`rank(−(1 − open / close))`
- 当日开盘价相对收盘价越低（阳线越长），因子值越大

In [8]:
plot_alpha_group(33)

### Alpha #41 — 几何均价偏离因子
`(high × low)^0.5 − vwap`
- 最高/最低价的几何均值与 VWAP 之差；当日内波动偏离均价时信号明显

In [9]:
plot_alpha_group(41)

### Alpha #55 — 价位与量排名反相关因子
`−corr(rank((close−ts_min(low,12))/(ts_max(high,12)−ts_min(low,12))), rank(volume), 6)`
- 衡量 12 日内价位高低与成交量排名的 6 日相关性，取反

In [10]:
plot_alpha_group(55)

### Alpha #101 — 日内涨跌幅占比因子
`(close − open) / ((high − low) + 0.001)`
- 实体部分占整个振幅的比例；阳线越长、上下影线越短，值越接近 1

In [11]:
plot_alpha_group(101)

## 5. 因子相关性矩阵

看看这几个因子在同一只股票上是否高度相关（若相关则存在信息冗余）。

In [12]:
import plotly.express as px

alpha_cols = [f"alpha{aid:03d}" for aid in ALPHA_IDS]

fig = make_subplots(
    rows=1, cols=len(stock_data),
    subplot_titles=[f"{STOCKS[c]}({c})" for c in stock_data],
    horizontal_spacing=0.04,
)

for idx, (code, df) in enumerate(stock_data.items()):
    corr = df[alpha_cols].corr()
    fig_heat = go.Heatmap(
        z=corr.values,
        x=[f"#{aid}" for aid in ALPHA_IDS],
        y=[f"#{aid}" for aid in ALPHA_IDS],
        colorscale="RdBu_r",
        zmin=-1, zmax=1,
        showscale=(idx == len(stock_data) - 1),
        text=corr.round(2).values,
        texttemplate="%{text}",
        textfont=dict(size=9),
    )
    fig.add_trace(fig_heat, row=1, col=idx + 1)

fig.update_layout(
    title="各股票 Alpha 因子相关性矩阵",
    height=400,
    width=1100,
    template="plotly_white",
)
fig.show()